# Programming Exercises

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
df_size = len(df)
disasters_size = len(df_train[df_train.target == 1])
print(f"Training set size: {df_size} (Distaters: {disasters_size}, Other: {df_size-disasters_size})")

df_test_size = len(df_test)
print(f"Testing set size: {df_test_size}")

Training set size: 7613 (Distaters: 3271, Other: 4342)
Testing set size: 3263


In [4]:
df.location[df.location.notna()].iloc[108]

'italy'

## 1
### a
7613 Training Points (3271 Disasters), 3263 Testing Points

In [6]:
# ID is uselesss and Location may bee to noisy
df_train = d_train.drop(columns=["id", "location"])

In [9]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words as englishwords
from nltk.stem import WordNetLemmatizer
def tokenize(string, lower=True, punctuation=True, stops=True, lemmatize=True, check_english=True):
    stop_words = set(stopwords.words("english"))
    english_words = set(englishwords.words())
    words = word_tokenize(string)
    lzr = WordNetLemmatizer() 

    if lower:
        words = [w.lower() for w in words]
    if punctuation:
        words = [w for w in words if w.isalnum()]
    if stops:
        words = [w for w in words if w not in stop_words]
    if lemmatize:
        words = [lzr.lemmatize(w) for w in words]
    # Takes long to run, use with caution
    if check_english:
        words = [w for w in words if w in english_words]
    return words

In [10]:
from nltk.stem import WordNetLemmatizer
lzr = WordNetLemmatizer()
df_train.text = df.text.apply(lambda x: tokenize(x))
df_train.keywords = df.keyword.apply(lambda x: x.lower() if isinstance(x, str) else x)

In [1]:
df_train.to_csv("new_train.csv")

NameError: name 'df' is not defined

In [101]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv("filtered_train.csv", index_col=False)

In [102]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, min_df=20)
sample = df.text.iloc[0]
X = vectorizer.fit_transform(df.text.values)
bag_of_words = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
df = pd.concat([df.drop(columns="Unnamed: 0"), bag_of_words], axis=1, join="inner")

In [103]:
X_train, X_dev, y_train, y_dev = train_test_split(df.drop(columns=["target", "keyword", "text"]), df.target, train_size=0.7, random_state=37)

### Logistic Regression

In [108]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=37).fit(X_train, y_train)
clf.score(X_dev, y_dev)

0.7784588441330998

### Linear SVM

In [109]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC(random_state=37).fit(X_train, y_train)
lsvm.score(X_dev, y_dev)

0.7753940455341506

### Non-Linear SVM

In [115]:
from sklearn.svm import SVC
lsvm = SVC(random_state=37, C=1).fit(X_train, y_train)
lsvm.score(X_dev, y_dev)

0.7832749562171629